In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

2023-04-25 22:13:09.088545: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class DataScraper:
    def __init__(self, years):
        self.years = years
        self.player_urls = [f'https://www.pro-football-reference.com/years/{year}/fantasy.htm' for year in self.years]
        self.team_url = 'https://www.pro-football-reference.com/years/{}/'

    def scrape_data(self, urls):
        data_frames = []
        for url in urls:
            year = url.split('/')[-2]  # Extract the year from the URL
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            dfs = pd.read_html(str(soup), flavor='bs4')
            df = pd.concat(dfs, ignore_index=True)
            df['Year'] = float(year)
            data_frames.append(df)

        return pd.concat(data_frames, ignore_index=True)

    def scrape_player_data(self):
        return self.scrape_data(self.player_urls)

    def scrape_team_data(self):
        team_urls = [self.team_url.format(year) for year in self.years]
        team_data = self.scrape_data(team_urls)
        return team_data

In [19]:
class DataPreprocessor:
    
    _dictionary = {'New York Giants': 'NYG',
 'Las Vegas Raiders': 'LVR',
 'Los Angeles Chargers': 'LAC',
 'Denver Broncos': 'DEN',
 'Green Bay Packers': 'GNB',
 'Jacksonville Jaguars': 'JAX',
 'Washington Redskins': 'WAS',
 'Los Angeles Rams': 'LAR',
 'Arizona Cardinals': 'ARI',
 'Carolina Panthers': 'CAR',
 'Baltimore Ravens': 'BAL',
 'New York Jets': 'NYJ',
 'Miami Dolphins': 'MIA',
 'Minnesota Vikings': 'MIN',
 'Oakland Raiders': 'OAK',
 'Chicago Bears': 'CHI',
 'New England Patriots': 'NWE',
 'Tennessee Titans': 'TEN',
 'New Orleans Saints': 'NOR',
 'Cleveland Browns': 'CLE',
 'Tampa Bay Buccaneers': 'TAM',
 'Buffalo Bills': 'BUF',
 'Cincinnati Bengals': 'CIN',
 'Houston Texans': 'HOU',
 'San Francisco 49ers': 'SFO',
 'Atlanta Falcons': 'ATL',
 'Washington Football Team': 'WAS',
 'Indianapolis Colts': 'IND',
 'Seattle Seahawks': 'SEA',
 'Pittsburgh Steelers': 'PIT',
 'Dallas Cowboys': 'DAL',
 'Detroit Lions': 'DET',
 'Philadelphia Eagles': 'PHI',
 'Kansas City Chiefs': 'KAN'}
    
    def __init__(self, data):
        self.data = data
        
    def preprocess_data(self):
        #Preprocess the data by calling helper methods in the correct order.
        self._convert_columns_to_float()
        self._flatten_multiindex_header()
        self._handle_missing_values()
        self._replace_team_names(self._dictionary)
        self._calculate_yards_per_play()
        return self.data
        
    def _convert_columns_to_float(self):
        # Convert columns to float if possible
        self.data = self.data.applymap(lambda x: pd.to_numeric(x, errors='ignore'))

    def _flatten_multiindex_header(self):
        # Flatten multi-level column index to single level column index if it exists
        if isinstance(self.data.columns, pd.MultiIndex):
            self.data.columns = ['_'.join(col).strip('_') if col[1] else col[0] for col in self.data.columns]
        return self.data

    def _calculate_yards_per_play(self):
        # Calculate yards per play for rushing and receiving
        if 'Rushing_Att' in self.data.columns and 'Rushing_Yds' in self.data.columns:
            self.data['Y/A'] = self.data.apply(lambda x: 0 if (pd.isna(x['Rushing_Att']) or x['Rushing_Att'] == 0) else x['Rushing_Yds'] / x['Rushing_Att'], axis=1)
        if 'Rec' in self.data.columns and 'Receiving_Yds' in self.data.columns:
            self.data['Y/R'] = self.data.apply(lambda x: 0 if (pd.isna(x['Rec']) or x['Rec'] == 0) else x['Receiving_Yds'] / x['Rec'], axis=1)
            
    def _handle_missing_values(self, thresh=0.5):
        # Drop rows where most of the columns have string data
        str_cols = self.data.select_dtypes(include=['object']).columns
        str_counts = self.data[str_cols].apply(lambda x: sum(x.apply(lambda y: isinstance(y, str))), axis=1)
        str_prop = str_counts / len(str_cols)
        self.data = self.data[str_prop <= thresh]

        # Fill applicable missing data
        should_fill_mask = (self.data.isnull() | (self.data == 0)).sum() / len(self.data) >= thresh
        cols_to_fill = should_fill_mask[should_fill_mask == True].index.tolist()
        self.data[cols_to_fill] = self.data[cols_to_fill].fillna(0)
        
         # Drop columns where most of the rows are null
        self.data.dropna(axis=1, thresh=len(self.data) * (1 - thresh), inplace=True)

        # Drop rows where most of the columns are null
        self.data.dropna(axis=0, thresh=len(self.data.columns) * thresh, inplace=True)

        # Drop rows where PPR data is null (if present in dataframe)
        if 'PPR' in self.data.columns:
            self.data.dropna(subset=['PPR'], inplace=True)

        # Drop rows where Rk > 400 if Rk is a column
        if 'Rk' in self.data.columns:
            self.data = self.data[self.data['Rk'] <= 400]
        
    def _replace_team_names(self, _dictionary):
        if 'Tm' not in self.data.columns:
            return self.data

        for key, value in _dictionary.items():
            mask = self.data['Tm'].str.startswith(key)
            if mask.any():
                self.data.loc[mask, 'Tm'] = value
        return self.data

In [20]:
class MergeAndProcess:
    
    def __init__(self, player_data, team_data):
        self.player_data = player_data
        self.team_data = team_data

    def merge(self):
        # Create a dictionary to map team data to new columns in player data
        team_data_map = {col: col if col in self.player_data.columns else f'Team_{col}' for col in self.team_data.columns if col not in ['Year', 'Tm']}

        # Create new columns in player data for team data
        self.player_data = self.player_data.assign(**{col: np.nan for col in team_data_map.values()})

        # Map team data to new columns in player data based on team and year
        self.player_data.set_index(['Year', 'Tm'], inplace=True)
        self.team_data.set_index(['Year', 'Tm'], inplace=True)
        self.player_data.update(self.team_data.rename(columns=team_data_map))

        # Flatten the column index
        if isinstance(self.player_data.columns, pd.MultiIndex):
            self.player_data.columns = self.player_data.columns.map(lambda x: x[1] if x[0].startswith('Team_') else x[0])

        return self.player_data.reset_index()
    
    def process(self):
        # Merge player and team data
        merged_data = self.merge()

        # Add next year PPR
        merged_data['next_year_PPR'] = merged_data.groupby('Player')['PPR'].shift(-1)

        # Add PPR per game
        merged_data['PPR_per_game'] = merged_data['PPR'] / merged_data['G']
        merged_data['PPR_per_game'].fillna(np.nan, inplace=True)

        # Convert column types
        for col in merged_data.select_dtypes(include=['object']).columns:
            try:
                merged_data[col] = pd.to_numeric(merged_data[col], errors='raise')
            except ValueError:
                merged_data[col] = merged_data[col].astype('string')

        # Drop missing values
        merged_data.dropna(inplace=True)

        # Reset index
        merged_data.reset_index(drop=True, inplace=True)

        # Replace non-alphanumeric characters in player names
        merged_data['Player'] = merged_data['Player'].str.replace(r'[^\w\s]+', '')

        return merged_data

In [21]:
years = list(range(2013, 2023))

In [22]:
scraper = DataScraper(years)
player_data = scraper.scrape_player_data()
team_data = scraper.scrape_team_data()

player_preprocessor = DataPreprocessor(player_data)
player_data = player_preprocessor.preprocess_data()

team_preprocessor = DataPreprocessor(team_data)
team_data = team_preprocessor.preprocess_data()

In [23]:
# create instance of MergeAndProcess class
merger = MergeAndProcess(player_data, team_data)

# call process_data method to merge and process data
merged_data = merger.process()

merged_data

KeyError: "None of ['Tm'] are in the columns"